In [1]:
!pip install gurobipy  # install gurobipy, if not already installed

     |████████████████████████████████| 11.5 MB 4.5 MB/s 


In [2]:
rom ast import Pass
from re import M
from turtle import shape
import networkx as nx
import gurobipy as gp
from gurobipy import GRB
import numpy as np

In [ ]:
class mid_mile:

    def __init__(self, K, L, M, R, N):

        #self.graph = graph
        self.K = K
        self.L = L                                       # no of legs
        self.M = M
        self.R = R
        self.N = N                                       # no of nodes/warehouses

        self.V = np.zeros(shape=(self.K))           # the demand of each commodity K in route R
        self.v = np.zeros(shape=(self.N, self.N, self.M))

        self.A = np.zeros(shape=(self.N, self.N, self.M))
        self.F = np.zeros(shape=(self.N, self.N, self.M))
        self.B = np.zeros(shape=(self.N, self.N, self.M))
        
        self.origins = []
        self.destinations = []
        self.routes = []


        self.C = np.zeros(shape=(self.R))
        self.matrix = np.zeros(shape=(self.N, self.N, self.R))

        self.model = gp.Model()
        
    def add_variables(self):
        f_list = []
        y_list = []
        rk = []

        # decision variables: xr, ylm, vlm, flm
        for n1 in range(self.N):
            for n2 in range(self.N):
                for m in range(self.M):
                    for k in range(self.K):
                        f_list.append((n1, n2, m, k))
                        for r in range(self.R):
                            y_list.append((n1, n2, m, r, k))

        for r in range(self.R):
            for k in range(self.K):
                rk.append((r,k))

        f_list = gp.tuplelist(f_list)
        y_list = gp.tuplelist(y_list)
        rk = gp.tuplelist(rk)

        self.x = self.model.addVars(rk, vtype=GRB.BINARY, name="x")
        self.y = self.model.addVars(y_list, vtype=GRB.BINARY, name="y")
        self.f = self.model.addVars(f_list, vtype=GRB.INTEGER, name="f")

        self.model.update()

    def add_init_constraints(self):
        self.add_constraints1()
        self.add_constraints2()
        self.add_constraints3()
        self.add_constraints4()
        self.add_constraints5()
        #self.obj_constraint()

    def add_constraints1(self):
        # # each commodity can have only one route associated only in the feasible set of routes that can take it from 'ok' to 'dk'
        # lhs = 0
        # for k in range(self.K):
        #     for r in range(self.R):
        #         # if r in self.routes[k]:                     #if r is a feasible route, then it contribute
        #         #     lhs += self.x[r, k]
        #         # else:
        #         #     self.x[r, k] = 0                        #if r is not a feasible route, then it should be zero
        #         lhs += self.x[r, k]
        #     self.model.addConstr(lhs == 1, name="c1")
        
        self.x[0, 0] = 1
        self.x[1, 0] = 0

        self.model.update()


    def add_constraints2(self):
        # constraint y[l, m, r, k]

        for r in range(self.R):
            for k in range(self.K):
                for n1 in range(self.N):
                    for n2 in range(self.N):
                        for m in range(self.M):    
                            rhs = self.x[r, k] * self.matrix[n1][n2][r]
                            lhs = self.y[n1, n2, m, r, k]
                            self.model.addConstr(lhs == rhs, name="c2")
        
        self.model.update()


    def add_constraints3(self):
        ###### each route can have atmost one commodity #######
        lhs = 0
        for r in range(self.R):
            for k in range(self.K):
                lhs += self.x[r, k]
            self.model.addConstr(lhs <= 1,name="c3")
        
        self.model.update()


    def add_constraints4(self):
        # summation of shipments should tally
        
        for k in range(self.K):
            #for r in self.routes[k]:
            for r in range(self.R):
                lhs = self.V[k] * self.x[r, k]
                for n1 in range(self.N):
                    for n2 in range(self.N):
                        rhs = 0
                        for m in range(self.M):
                            rhs += self.v[n1][n2][m] * self.f[n1, n2, m, k] * self.y[n1, n2, m, r, k]
                            #rhs += self.v[n1][n2][m] * self.f[n1, n2, m, k] * self.matrix[n1][n2][r]
                            
                        
                        name_c4 = "c4_{}{}{}{}".format(n1, n2, r, k)
                        self.model.addConstr(lhs - rhs <= 0, name=name_c4)
        
        self.model.update()


    def add_constraints5(self):
        for n1 in range(self.N):
            for n2 in range(self.N):
                for m in range(self.M):
                    lhs = 0
                    for k in range(self.K):
                        lhs += self.f[n1, n2, m, k]
                    #rhs = self.F[n1][n2][m] * self.y[l, m]
                    self.model.addConstr(lhs - 10 <= 0, name="c5")
            
        self.model.update()
        pass
    

    def add_constraint6(self):
        for k in range(self.K):
            for r in range(self.R):


    # def obj_constraint(self):
    #     lhs = 0
    #     for e in self.edges:
    #         lhs += self.graph[e[0]][e[1]]['length'] * self.x[a, e[0], e[1]]
    #     #this constraint ensures that c is atleast equal to the max of the walks of the agents
    #     print(self.c)
    #     self.model.addConstr(lhs <= self.c[0], name="obj_agent{}".format(a))
        
    #     self.model.update()


    def objective(self):
        term2 = 0
        for n1 in range(self.N):
            for n2 in range(self.N):
                for m in range(self.M):
                    for k in range(self.K):
                        term2 += self.A[n1][n2][m]*self.f[n1, n2, m, k]
        
        term1 = 0
        for k in range(self.K):
            for r in range(self.R):
                term1 += self.x[r, k] * self.C[r]
        
        obj = term1 + term2

        self.model.setObjective(obj, GRB.MINIMIZE)

    def process(self):
        self.status = self.model.optimize()

    def get_solution(self):
        pass
    

    # this functions is wrong; because it shows the route only if the ok and dk are directly connected -- we may have to implement standard path planning algorithms
    def get_routes(self):
        for k in range(self.K):
            rs = []
            ok = self.origins[k]
            dk = self.destinations[k]
            
            for r in range(self.R):
                if self.matrix[ok][dk][r] == 1:
                    rs.append(r)
            
            self.routes.append(rs)

In [ ]:
# Input model parameter values

K = 1
M = 1                       # only one mode of transport
N = 3
R = 2
L = 3


origins_k = [0]
destinations_k = [2]

V = np.zeros(shape=(K))
V[0] = 10


######## to be implemented ========>>>>>>>>>>>>> it can also depend on mode of transport ###########
C = [10, 5]

v = np.zeros(shape=(N, N, M))
v[0][1][0] = 5
v[1][2][0] = 7
v[0][2][0] = 6


F = np.zeros(shape=(N, N, M))
F[0][1][0] = 5
F[1][2][0] = 3
F[0][2][0] = 4


A = np.ones(shape=(N, N, M))
A[0][1][0] = 9
A[1][2][0] = 2
A[0][2][0] = 6


graph = np.zeros(shape=(N, N, R))
graph[0][1][0] = 1
graph[1][2][0] = 1
graph[0][2][1] = 1

In [ ]:
test = mid_mile(K, L, M, R, N)

test.V = V
test.v = v
test.F = F
test.A = A
test.C = C
test.matrix = graph
test.origins = origins_k
test.destinations = destinations_k

test.routes = [[0,1]]



test.add_variables()

test.add_constraints1()
test.add_constraints2()
test.add_constraints3()
#test.add_constraints4()
test.add_constraints5()


test.objective()
test.process()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 29 rows, 29 columns and 27 nonzeros
Model fingerprint: 0x6d3e4234
Variable types: 0 continuous, 29 integer (20 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 9e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Found heuristic solution: objective 10.0000000

Explored 0 nodes (0 simplex iterations) in 0.03 seconds (0.00 work units)
Thread count was 1 (of 2 available processors)

Solution count 1: 10 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.000000000000e+01, best bound 1.000000000000e+01, gap 0.0000%


In [ ]:
print(test.routes)

[[0, 1]]


In [ ]:
y_sol = test.model.getAttr('X', test.y)
f_sol = test.model.getAttr('X', test.f)
#x_sol = test.model.getAttr('X', test.x)

AttributeError: ignored

In [ ]:
#x_sol = test.model.getAttr('X', test.x)
print(test.x)

{(0, 0): 1, (1, 0): 0}


In [ ]:
for k in range(K):
    for n1 in range(N):
        for n2 in range(N):
            for m in range(M):
                if f_sol[n1, n2, m, k] == 0:
                    print((n1, n2, m , k) + " : {}".format(f_sol[n1, n2, m, k]))

TypeError: ignored

In [ ]:
print(f_sol)

{(0, 0, 0, 0): 0.0, (0, 1, 0, 0): 0.0, (0, 2, 0, 0): 0.0, (1, 0, 0, 0): 0.0, (1, 1, 0, 0): 0.0, (1, 2, 0, 0): 0.0, (2, 0, 0, 0): 0.0, (2, 1, 0, 0): 0.0, (2, 2, 0, 0): 0.0}


In [ ]:
for r in range(R):
    for n1 in range(N):
        for n2 in range(N):
            for m in range(M):
                for k in range(K):
                    if y_sol[n1, n2, m, r, k] == 1:
                        print((n1, n2, m , r, k))

(0, 1, 0, 0, 0)
(1, 2, 0, 0, 0)


# Diagnosis

In [ ]:
# do IIS
print('The model is infeasible; computing IIS')
test.model.computeIIS()
if test.model.IISMinimal:
    print('IIS is minimal\n')
else:
    print('IIS is not minimal\n')
print('\nThe following constraint(s) cannot be satisfied:')
for c in test.model.getConstrs():
    if c.IISConstr:
        print('%s' % c.ConstrName)

The model is infeasible; computing IIS

Computing Irreducible Inconsistent Subsystem (IIS)...

           Constraints          |            Bounds           |  Runtime
      Min       Max     Guess   |   Min       Max     Guess   |
--------------------------------------------------------------------------
        0        45         -         0         9         -           0s
        1         1         -         0         0         -           0s

IIS computed: 1 constraints, 0 bounds
IIS runtime: 0.02 seconds (0.00 work units)
IIS is minimal


The following constraint(s) cannot be satisfied:
